In [169]:
# TODO: join dataframes, perform regression; merge and resample carbon data into one raster, 

In [193]:
import sys
import os
import subprocess
import datetime
import pandas as pd
import numpy as np
import pygeoprocessing
from osgeo import gdal

In [204]:
def get_raster_resolution(raster):
    """Prints out dimensions of raster. 
    Returns pixel dimensions in tuple (pixelSizeX, pixelSizeY).
    
    Keyword arguments:
    raster -- GDAL-loaded raster to measure
    """
    
    # NOTE: GetGeoTransform fetches the coefficients for transforming between 
    # pixel/line (P,L) raster space, and projection coordinates (Xp,Yp) space.
    # Xp = padfTransform[0] + P*padfTransform[1] + L*padfTransform[2];
    # Yp = padfTransform[3] + P*padfTransform[4] + L*padfTransform[5];
    gt = raster.GetGeoTransform()
    print("Geotransform is", gt) 
    
    pixelSizeX = gt[1]
    pixelSizeY =-gt[5]
    pixelDims = (pixelSizeX, pixelSizeY)
    
    print("Where (pixelSizeX, pixelSizeY) is", pixelDims)
    
    return pixelDims

def resample_raster_to_resolution(raster_src_path, xres, yres):
    """Uses gdalwarp to resample raster via averaging to the given resolution. 
    
    Keyword arguments:
    raster_src_path -- (dtype: string) file path to raster to resample
    xres -- (dtype: int) desired x-resolution
    yres -- (dtype: int) desired y-resolution
    """
    
    # Get base filename without extension
    filename = os.path.basename(raster_src_path)[:-4]
    
    # Denote resolution of 5 arcmin to file name; alternatively, call it _resampled    
    resampled_dest_path = os.path.join(OUTPUT_DIR, filename + "_5m.tif")
    
    # Can specify different resampling method (-r) depending on purpose
    args = ['gdalwarp', '-tr', str(xres), str(yres), '-r', 'average', raster_src_path, resampled_dest_path]
    
    print("Resampling raster file...start time:", datetime.datetime.now())
    
    # Call system command to run GDAL resampling
    process = subprocess.check_output(args)
    
    return resampled_dest_path
    
def convert_raster_to_xyz(raster_src_path):
    """Converts given raster to .xyz format using gdal_translate.
    
    Keyword arguments:
    raster_src_path -- (dtype: string) file path to .tif file to convert
    """
    
    # Remove 3-letter tif extension, replace with XYZ extension
    filename = os.path.basename(raster_src_path)[:-4]
    xyz_filename = ('{}.xyz').format(filename)
    
    xyz_dest_path = os.path.join(OUTPUT_DIR, xyz_filename)
    
    # Arguments for GDAL translation
    args = ["gdal_translate", raster_src_path, xyz_dest_path, "-co", "ADD_HEADER_LINE=YES"]
    
    print("Now converting raster into XYZ format...start time:", datetime.datetime.now())
    
    # Call system command to run GDAL translation
    process = subprocess.check_output(args)
    
    print("Saved new XYZ file to", xyz_dest_path)
    return xyz_dest_path

def convert_xyz_to_df(xyz_src_path, value_col_name):
    """Reads XYZ file into a dataframe with columns {"pixel_id", "lat", "long", "value"}.
    Returns this dataframe. 
    
    Keyword arguments:
    xyz_src_path -- (dtype: string) file path to .xyz file to read in
    """
    
    print("Now reading file into dataframe...start time:", datetime.datetime.now())
    df = pd.read_csv(xyz_src_path, delimiter=' ')
    df.index.name = 'pixel_id'
    df.rename(columns={'X': 'lat', 'Y': 'long', 'Z': value_col_name}, inplace=True)
    print(df.head)
    return df

def convert_raster_to_df(tif_src_path, value_col_name="value"):
    """Reads a raster file into a dataframe with columns {"pixel_id", "lat", "long", "value"}
    Returns this dataframe. 
    
    Keyword arguments:
    xyz_src_path -- (dtype: string) file path to raster file to read in
    """
    dest_path = convert_raster_to_xyz(tif_src_path)
    new_df = convert_xyz_to_df(dest_path, value_col_name)
    return new_df

In [195]:
# Define working directories
NAT_CAP_DIR = '/Users/jackieennis/Google Drive/Classes/Active Classes/Impact Lab/Carbon Students Project'
CARBON_DIR = os.path.join(NAT_CAP_DIR, 'Carbon')
OUTPUT_DIR = os.path.join(CARBON_DIR, 'Outputs')
PEOPLE_DIR = os.path.join(NAT_CAP_DIR, 'People')

test_foldername = 'test_NE1_50M_SR'
test_filename = 'NE1_50M_SR.tif'
path_to_test_raster = os.path.join(CARBON_DIR, test_foldername, test_filename)

market_dist_foldername = 'Distance_to_market'
market_dist_filename = 'minutes_to_market_5m.tif'
path_to_market_dist_raster = os.path.join(PEOPLE_DIR, market_dist_foldername, market_dist_filename)

In [196]:
# Find desired resolution for 5 arcmins
mins_to_market_raster = gdal.Open(path_to_market_dist_raster)
(Xres, Yres) = get_raster_resolution(mins_to_market_raster)

# Resample the larger raster down to 5arcmin
path_to_test_raster_resampled = resample_raster_to_resolution(path_to_test_raster, Xres, Yres)

Geotransform is (-180.0, 0.083333333333333, 0.0, 89.99999999928002, 0.0, -0.083333333333333)
Where (pixelSizeX, pixelSizeY) is (0.083333333333333, 0.083333333333333)
Resampling raster file...start time: 2019-05-16 10:57:08.459420


In [197]:
# Load new resampled raster
test_raster = gdal.Open(path_to_test_raster_resampled)
get_raster_resolution(test_raster)

Geotransform is (-179.99999999999997, 0.083333333333333, 0.0, 90.0, 0.0, -0.083333333333333)
Where (pixelSizeX, pixelSizeY) is (0.083333333333333, 0.083333333333333)


(0.083333333333333, 0.083333333333333)

In [201]:
# Read rasters into dataframes. This takes several minutes - be patient and only re-run when necessary.
test_df = convert_raster_to_df(path_to_test_raster_resampled)
# market_dist_df = convert_raster_to_df(path_to_market_dist_raster)                           

Now converting raster into XYZ format...start time: 2019-05-16 11:07:12.150478
Saved new XYZ file to /Users/jackieennis/Google Drive/Classes/Active Classes/Impact Lab/Carbon Students Project/Carbon/Outputs/NE1_50M_SR_5m.xyz
Now reading file into dataframe...start time: 2019-05-16 11:07:37.800050
<bound method NDFrame.head of                  lat       long  value
pixel_id                              
0        -179.958333  89.958333    251
1        -179.875000  89.958333    251
2        -179.791667  89.958333    251
3        -179.708333  89.958333    251
4        -179.625000  89.958333    251
5        -179.541667  89.958333    251
6        -179.458333  89.958333    251
7        -179.375000  89.958333    251
8        -179.291667  89.958333    251
9        -179.208333  89.958333    251
10       -179.125000  89.958333    251
11       -179.041667  89.958333    251
12       -178.958333  89.958333    251
13       -178.875000  89.958333    251
14       -178.791667  89.958333    251
15       -

In [211]:
print(test_df)
print(test_df.shape)
print(market_dist_df)      
print(market_dist_df.shape)

                 lat       long  value
pixel_id                              
0        -179.958333  89.958333    251
1        -179.875000  89.958333    251
2        -179.791667  89.958333    251
3        -179.708333  89.958333    251
4        -179.625000  89.958333    251
5        -179.541667  89.958333    251
6        -179.458333  89.958333    251
7        -179.375000  89.958333    251
8        -179.291667  89.958333    251
9        -179.208333  89.958333    251
10       -179.125000  89.958333    251
11       -179.041667  89.958333    251
12       -178.958333  89.958333    251
13       -178.875000  89.958333    251
14       -178.791667  89.958333    251
15       -178.708333  89.958333    251
16       -178.625000  89.958333    251
17       -178.541667  89.958333    251
18       -178.458333  89.958333    251
19       -178.375000  89.958333    251
20       -178.291667  89.958333    251
21       -178.208333  89.958333    251
22       -178.125000  89.958333    251
23       -178.041667  89.

In [217]:
merged_df = test_df.merge(market_dist_df, on=['pixel_id', 'lat', 'long'], suffixes=('_color', '_mins'))

In [219]:
merged_df1 = test_df.merge(market_dist_df, on=['pixel_id'], suffixes=('_color', '_mins'))

In [220]:
merged_df1

,lat_color,long_color,value_color,lat_mins,long_mins,value_mins
pixel_id,,,,,,
0,-179.958333,89.958333,251,-179.958333,89.958333,-2147483647
1,-179.875000,89.958333,251,-179.875000,89.958333,-2147483647
2,-179.791667,89.958333,251,-179.791667,89.958333,-2147483647
3,-179.708333,89.958333,251,-179.708333,89.958333,-2147483647
4,-179.625000,89.958333,251,-179.625000,89.958333,-2147483647
5,-179.541667,89.958333,251,-179.541667,89.958333,-2147483647
6,-179.458333,89.958333,251,-179.458333,89.958333,-2147483647
7,-179.375000,89.958333,251,-179.375000,89.958333,-2147483647
8,-179.291667,89.958333,251,-179.291667,89.958333,-2147483647
